In [1]:
import sqlite3
from sqlite3 import Error
from sqlite3 import IntegrityError
from ipyfilechooser import FileChooser
import pandas as pd
import io
import requests
import numpy
from scipy import stats
import math
from scipy.stats import ttest_ind
import tqdm

In [2]:
starting_directory = '/Users/Joseph/Desktop'
baseline_chooser = FileChooser(starting_directory)
display(baseline_chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [3]:
tap_chooser=FileChooser(starting_directory)
display(tap_chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [116]:
Screen='PD_Screen' #Refer to Key
folder_path=baseline_chooser.selected_path
print(folder_path)

/Users/lavanya/Desktop/Lavanya_Test/Batch_1


In [117]:
baseline_output = pd.read_csv(baseline_chooser.selected, index_col=0).drop(columns=['index'])
print(baseline_output.head())
print(baseline_output.shape)
# X=baseline_output.drop(columns=['Time','n','Number','Tap','Puff','x','y','plate','dataset','Gene','Allele'])
# print(X)

    Time   n  Number  Instantaneous Speed  Interval Speed   Bias  Morphwidth  \
0  490.0  77      40               0.0417          0.0581  0.100      0.1140   
1  490.0  76      40               0.0510          0.0729  0.100      0.1152   
2  490.0  77      40               0.0589          0.0864  0.100      0.1139   
3  490.0  77      40               0.0445          0.0662  0.075      0.1140   
4  490.2  77      40               0.0452          0.0656  0.100      0.1155   

   Midline    Area  Angular Speed  ...  Kink  Curve    Crab  Pathlength  \
0    1.095  0.1472            1.6  ...  40.8   27.0  0.0041       10.33   
1    1.093  0.1483            2.2  ...  40.0   26.5  0.0049       10.33   
2    1.090  0.1462            2.5  ...  39.5   26.8  0.0058       10.33   
3    1.089  0.1464            2.0  ...  40.4   27.3  0.0048       10.33   
4    1.094  0.1482            2.2  ...  40.9   27.3  0.0059       10.33   

   Plate_id      Date     Screen dataset Gene Allele  
0   B0520ab  

In [118]:
tap_output = pd.read_csv(tap_chooser.selected, index_col=0)
print(tap_output.head())

      time  dura   dist      prob     speed  plate      Date Plate_id  \
0  599.983  2.64  0.739  0.945946  0.279924      1  20220524  B0520ba   
1  609.995  2.66  0.834  0.857143  0.313534      1  20220524  B0520ba   
2  619.933  2.48  0.759  0.833333  0.306048      1  20220524  B0520ba   
3  629.932  2.19  0.649  0.901961  0.296347      1  20220524  B0520ba   
4  639.991  2.06  0.589  0.787234  0.285922      1  20220524  B0520ba   

      Screen  taps        dataset     Gene Allele  
0  PD_Screen   1.0  F53B2.5_ok226  F53B2.5  ok226  
1  PD_Screen   2.0  F53B2.5_ok226  F53B2.5  ok226  
2  PD_Screen   3.0  F53B2.5_ok226  F53B2.5  ok226  
3  PD_Screen   4.0  F53B2.5_ok226  F53B2.5  ok226  
4  PD_Screen   5.0  F53B2.5_ok226  F53B2.5  ok226  


In [119]:
PD_tap=tap_output
PD_first_tap = PD_tap[(PD_tap.taps==1)].reset_index().drop(columns="index")
PD_last_three_taps = PD_tap[((PD_tap.taps>=28)&(PD_tap.taps<=30))].reset_index().drop(columns="index")
PD_recov_taps = PD_tap[(PD_tap.taps==31)].reset_index().drop(columns="index")

print(PD_first_tap)
print("-"*55)
print(PD_last_three_taps)
print("-"*55)

print("-"*55)
PD_final_taps=PD_last_three_taps.groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"]).mean().reset_index()
PD_final_taps=PD_final_taps.rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")


print("PD_final_taps")
print(PD_final_taps)
print("-"*55)


PD_first_tap=PD_first_tap.rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
PD_habit_levels = pd.merge(PD_first_tap, PD_final_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
PD_habit_levels = PD_habit_levels.dropna()

print('PD_habit_levels')
print(PD_habit_levels)
print("-"*55)
PD_habit_levels['habit_dura']=PD_habit_levels.init_dura-PD_habit_levels.final_dura
PD_habit_levels['habit_prob']=PD_habit_levels.init_prob-PD_habit_levels.final_prob
PD_habit_levels['habit_speed']=PD_habit_levels.init_speed-PD_habit_levels.final_speed
# PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y', 'Allele','Gene'])
PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y'])

print('final version PD_habit_levels')
print(PD_habit_levels)
print("-"*55)

PD_recov_taps=PD_recov_taps.rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
print(PD_recov_taps)
print("-"*55)
if PD_recov_taps.empty:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='outer')
else:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
print('After checking recov_taps empty')
print(PD_habituation)

PD_habituation = PD_habituation.drop(columns=['taps_x','taps_y'])
PD_habituation = PD_habituation.dropna() #remove for G-Protein
print('After drop na')
print(PD_habituation)

PD_habituation['recovery_dura']=(PD_habituation.recov_dura-PD_habituation.init_dura)/PD_habituation.init_dura*100
PD_habituation['recovery_prob']=(PD_habituation.recov_prob-PD_habituation.init_prob)/PD_habituation.init_prob*100
PD_habituation['recovery_speed']=(PD_habituation.recov_speed-PD_habituation.init_speed)/PD_habituation.init_speed*100
tap_data=PD_habituation.dropna() #remove for G-Protein
#uncomment below for G-Protein
# tap_data=PD_habituation.dropna(subset = ['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
#        'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
#        'final_speed', 'habit_dura', 'habit_prob', 'habit_speed'])

print("-"*55)
print(tap_data)

print("-"*55)


        time  dura   dist      prob     speed  plate      Date Plate_id  \
0    599.983  2.64  0.739  0.945946  0.279924      1  20220524  B0520ba   
1    599.951  2.84  0.921  0.921569  0.324296      2  20220524  B0520bd   
2    599.967  3.64  1.246  0.947368  0.342308      3  20220524  A0520be   
3    599.984  3.32  1.158  0.925000  0.348795      4  20220524  A0520bc   
4    599.953  3.02  1.003  0.883721  0.332119      5  20220524  C0520bb   
..       ...   ...    ...       ...       ...    ...       ...      ...   
157  599.967  2.86  0.362  0.666667  0.126573      1  20220513  B0509ba   
158  599.962  3.52  0.435  0.473684  0.123580      2  20220513  B0509bd   
159  599.978  2.67  0.605  0.250000  0.226592      3  20220513  C0509bc   
160  599.960  3.39  0.535  0.173913  0.157817      4  20220513  A0509bb   
161  599.986  4.36  0.639  0.192308  0.146560      5  20220513  A0509be   

        Screen  taps        dataset     Gene Allele  
0    PD_Screen   1.0  F53B2.5_ok226  F53B2.5 

### The below cell reads in file from online repository (github) - example in case this is possible in the future

In [120]:
# # tap_url = 'https://osf.io/du9bj/files/osfstorage/650a2f9f1e76a4230e8a99a5?raw=true'
# tap_url='https://github.com/MyYummyPancake/NRSC510B/blob/main/tap_output.csv?raw=true'
# # s=requests.get(tap_url).content
# # tap_output=pd.read_csv(io.StringIO(s.decode('utf-8')))
# tap_output=pd.read_csv(tap_url, on_bad_lines='skip', index_col=0)
# print(tap_output)

In [121]:
# print(tap_output['Gene'].unique())
# print(len(tap_output['Gene'].unique()))
# print(baseline_output['Gene'].unique())
# print(len(baseline_output['Gene'].unique()))

In [122]:
# for x in tap_output['Gene'].unique():
#     tap_output_gene=tap_output[tap_output['Gene']==x]
#     gene_tap_data=tap_output[tap_output['Date'].isin(tap_output_gene['Date'].unique())]
#     gene_tap_data_final=gene_tap_data[gene_tap_data['Gene'].isin(['N2', x])]
#     gene_tap_data_final['taps']=gene_tap_data_final['taps'].astype(int)


# for x in baseline_output['Gene'].unique():
#     baseline_output_gene=baseline_output[baseline_output['Gene']==x]
#     gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
#     gene_baseline_data_final=gene_baseline_data[gene_baseline_data['Gene'].isin(['N2', x])]
#     for a,b in zip(list_baseline_metrics, list_baseline_Tstats):
#         TTest_Allele(x, a, baseline_output, b)


# Run Statistics (T-Test and sample-mean distance) on Data

## Sample Mean Distance & Confidence Interval Calculation of Baseline Data
1. Calculate Mean

2. Calculate SEM

3. Calculate CI

4. Calculate Mean Distance From Control (Mean of Strain - Mean of Control)

In [123]:
print(tap_data.columns)

Index(['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed', 'time',
       'recov_dura', 'dist', 'recov_prob', 'recov_speed', 'taps',
       'recovery_dura', 'recovery_prob', 'recovery_speed'],
      dtype='object')


In [124]:
print(tap_data.head())

   init_dura  init_prob  init_speed  plate      Date Plate_id     Screen  \
0       2.64   0.945946    0.279924      1  20220524  B0520ba  PD_Screen   
1       2.84   0.921569    0.324296      2  20220524  B0520bd  PD_Screen   
2       3.64   0.947368    0.342308      3  20220524  A0520be  PD_Screen   
3       3.32   0.925000    0.348795      4  20220524  A0520bc  PD_Screen   
4       3.02   0.883721    0.332119      5  20220524  C0520bb  PD_Screen   

         dataset     Gene Allele  ...  habit_speed      time  recov_dura  \
0  F53B2.5_ok226  F53B2.5  ok226  ...     0.083399  1189.940        3.72   
1  F53B2.5_ok226  F53B2.5  ok226  ...     0.107957  1189.943        3.11   
2  F53B2.5_ok226  F53B2.5  ok226  ...     0.129027  1189.970        3.65   
3  F53B2.5_ok226  F53B2.5  ok226  ...     0.086841  1189.961        2.62   
4  F53B2.5_ok226  F53B2.5  ok226  ...     0.114514  1189.990        2.90   

    dist  recov_prob  recov_speed  taps  recovery_dura  recovery_prob  \
0  1.090    0

In [125]:
def get_output_byplate(output, baseline, allele):
    drop_col_base=['Plate_id','n','Number','Time','Screen','Date','Allele']
    drop_col_taps=['Plate_id','Screen','Date','Allele','dist','plate','time',
                   'taps','recov_dura','recov_prob','recov_speed']
    drop_col = drop_col_base if baseline else drop_col_taps
    drop_col.append('Gene') if allele else drop_col.append('dataset')
     
    output_byplate = output.groupby(
        by=['Plate_id','Date','Screen','dataset','Gene','Allele'],
        as_index=False).mean().drop(columns=drop_col)
    
    return output_byplate


baseline_output_byplate=get_output_byplate(baseline_output, baseline= True ,allele=False)
print(baseline_output_byplate)
print('-'*30)
print(baseline_output_byplate.columns[1:])

print('-'*30)
tap_data_byplate=get_output_byplate(tap_data, baseline=False, allele=False)


print(tap_data_byplate)
print('-'*30)
print(tap_data_byplate.columns)
print('-'*50)

print("allele output")
print('-'*50)

baseline_output_allele_byplate=get_output_byplate(baseline_output,baseline=True, allele=True)
print(baseline_output_allele_byplate)
print('-'*50)
print(baseline_output_allele_byplate.columns)
# print(baseline_output_byplate[['Gene','Instantaneous Speed']].copy())

print('-'*50)
tap_data_allele_byplate= get_output_byplate(tap_data, baseline=False, allele=True)

print(tap_data_allele_byplate)
print('-'*50)
print(tap_data_allele_byplate.columns)

       Gene  Instantaneous Speed  Interval Speed      Bias  Morphwidth  \
0        N2             0.044749        0.050152  0.061512    0.105066   
1        N2             0.066769        0.100861  0.093839    0.114252   
2     rme-8             0.042439        0.035830  0.037630    0.058271   
3     rme-8             0.040744        0.032651 -0.009157    0.053671   
4        N2             0.050556        0.049911  0.055680    0.108573   
..      ...                  ...             ...       ...         ...   
154   gba-3             0.087495        0.082014  0.273561    0.121294   
155  sipa-1             0.078053        0.087867  0.189327    0.144883   
156  let-19             0.076868        0.073004  0.227203    0.119369   
157   lrk-1             0.041429        0.028091  0.006531    0.119431   
158  let-19             0.072514        0.067091  0.183739    0.112399   

      Midline      Area  Angular Speed  Aspect Ratio       Kink      Curve  \
0    1.144583  0.142881       2.6

In [126]:
# print(tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_XJ1'])

In [127]:

def extract_phenotypes(df):
    ''' 
    input: one pd.DataFrame with multiple columns (1st column is the index, the other are phenotypes)
    returns: list of pd.DataFrame, each DataFrame with 2 columns: index and 1 phenotype, 
            for how many phenotypes there are in the input
    '''
    list_phenotypes_df = []
    phenotypes = df.columns[1:]
    index = df.columns[0]
    for i in phenotypes:
        new_df = df[[index, i]].copy()
        list_phenotypes_df.append(new_df)
    return list_phenotypes_df


# baseline_output_gene=baseline_output[baseline_output['dataset']==x]
# gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
# gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]

def calculate_MSD(list_of_dfs, by):
    '''
    input: list of dataframes with 2 columns (1st one is index, 2nd one are indiviudal phenotypes)
            output from extract_phenotypes function

    returns: the same list of dataframes with additional calculated columns for:
                    1. mean
                    2. count
                    3. std
                    4. ci95_hi
                    5. ci95_low
                    6. ci95
    '''
    new_list_of_dfs=[]
    for i in list_of_dfs:
        calculate_1= i.groupby(by=[by], as_index=False).agg(['mean', 'count', 'sem'])
        calculate_2= ci95(calculate_1)
        N2_data = calculate_2[calculate_2.index=='N2']
        calculate_2.iloc[:,0] = calculate_2.iloc[:,0]-N2_data.iloc[0,0]
        calculate_2.iloc[:,3] = calculate_2.iloc[:,3]-N2_data.iloc[0,0]
        calculate_2.iloc[:,4] = calculate_2.iloc[:,4]-N2_data.iloc[0,0]
        new_list_of_dfs.append(calculate_2)
    return new_list_of_dfs


def ci95(df):
    """
    input: df of 4 columns: index, mean, count, std

    returns: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

    """
    for metric in df.columns.levels[0]:
        if metric == 'Gene':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m, c, s = df[metric].loc[i]
                ci95_hi.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
    return df


def get_MSD(list_MSD):
    '''
    input: List of dataframes, each representing a phenotype with calculated MSD.

    returns: Single combined dataframe joining all input dataframes with MSD values.
    '''
    for a in list_MSD:
        if a.columns.levels[0] == list_MSD[0].columns.levels[0]:
            MSD=a
        else:
            MSD=MSD.join(a)
    return MSD



In [128]:
def get_combined_MSD(baseline_byplate,tap_byplate, by=['Gene','dataset']):
    """
    Combines MSD datafram from baseline plates and tap plates

    input:
        - baseline_byplate: baseline data by plate
        - tap_byplate: tap data by plate
        - by: what to group by "Gene" or "dataset"
    returns:
        - combined MSD dataframe
    """
    list_baseline_MSD=calculate_MSD(extract_phenotypes(baseline_byplate), by=by)

    list_tap_MSD=calculate_MSD(extract_phenotypes(tap_byplate), by=by)

    baseline_MSD = get_MSD(list_baseline_MSD)
    
    tap_MSD = get_MSD(list_tap_MSD)

    combined_MSD = baseline_MSD.join(tap_MSD, on=by)

    combined_MSD=combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                         "habit_prob": "Habituation of Respones Probability",
                                         "habit_speed":"Habituation of Response Speed",
                                         "init_dura": "Initial Response Duration",
                                         "init_prob": "Initial Response Probability",
                                         "init_speed": "Initial Response Speed",
                                         "final_dura": "Final Response Duration",
                                         "final_prob": "Final Response Probability",
                                         "final_speed": "Final Response Speed",
                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                         "recovery_speed": "Spontaneous Recovery of Response Speed"})

    combined_MSD=combined_MSD.reset_index()
    combined_MSD.columns = combined_MSD.columns.to_flat_index().str.join('-')
    combined_MSD=combined_MSD.rename(columns={by+"-": by})
    combined_MSD['Screen']=Screen
    
    return combined_MSD

## Gene-level SMD

In [129]:
combined_MSD=get_combined_MSD(baseline_output_byplate,
                              tap_data_byplate, 
                              by='Gene')

In [130]:
print(combined_MSD)

       Gene  Instantaneous Speed-mean  Instantaneous Speed-count  \
0   F53B2.5                  0.086278                          5   
1        N2                  0.000000                         40   
2     cat-1                  0.065821                          5   
3     cat-2                  0.055155                         15   
4    catp-7                 -0.013505                          5   
5     cpr-1                  0.004338                          5   
6    crls-1                 -0.020205                          5   
7     dat-1                  0.029433                          5   
8     dop-1                  0.012138                          5   
9     dop-3                  0.042760                          5   
10    gba-3                  0.002792                          5   
11    gly-5                 -0.001453                          4   
12    ifg-1                  0.019292                          5   
13   immt-1                  0.002092           

## Allele-level SMD

In [131]:
allele_combined_MSD=get_combined_MSD(baseline_output_allele_byplate,
                                     tap_data_allele_byplate, 
                                     by='dataset')

In [132]:
print(allele_combined_MSD)

            dataset  Instantaneous Speed-mean  Instantaneous Speed-count  \
0     F53B2.5_ok226                  0.086278                          5   
1                N2                  0.000000                         40   
2       cat-1_ok411                  0.065821                          5   
3       cat-2_e1112                  0.068617                          5   
4       cat-2_n4547                  0.062138                          5   
5      cat-2_tm2261                  0.034711                          5   
6     catp-7_tm8556                 -0.013505                          5   
7      cpr-1_ok1344                  0.004338                          5   
8     crls-1_tm2575                 -0.020205                          5   
9       dat-1_ok157                  0.029433                          5   
10      dop-1_vs101                  0.012138                          5   
11      dop-3_ok295                  0.042760                          5   
12     gba-3

## T-Stat analysis

In [133]:
def baseline_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for baseline analysis

    input:
        - by: what to group by "Gene" or "dataset"
    returns:
        - list_baseline_Tstats: to store t-statistics
        - list_baseline_metrics: to store metic names
    """
    PD_baseline_instantspeed_T=pd.DataFrame(columns = [by,"Instantaneous Speed"])
    PD_baseline_intspeed_T=pd.DataFrame(columns = [by,"Interval Speed"])
    PD_baseline_bias_T=pd.DataFrame(columns = [by,"Bias"])
    PD_baseline_morphwidth_T=pd.DataFrame(columns = [by,"Morphwidth"])
    PD_baseline_midline_T=pd.DataFrame(columns = [by,"Midline"])
    PD_baseline_area_T=pd.DataFrame(columns = [by,"Area"])
    PD_baseline_angularspeed_T=pd.DataFrame(columns = [by,"Angular Speed"])
    PD_baseline_aspectratio_T=pd.DataFrame(columns = [by,"Aspect Ratio"])
    PD_baseline_kink_T=pd.DataFrame(columns = [by,"Kink"])
    PD_baseline_curve_T=pd.DataFrame(columns = [by,"Curve"])
    PD_baseline_crab_T=pd.DataFrame(columns = [by,"Crab"])
    PD_baseline_pathlength_T=pd.DataFrame(columns = [by,"Pathlength"])

    list_baseline_Tstats=[PD_baseline_instantspeed_T,
                        PD_baseline_intspeed_T,
                        PD_baseline_bias_T,
                        PD_baseline_morphwidth_T,
                        PD_baseline_midline_T,
                        PD_baseline_area_T,
                        PD_baseline_angularspeed_T,
                        PD_baseline_aspectratio_T,
                        PD_baseline_kink_T,
                        PD_baseline_curve_T,
                        PD_baseline_crab_T,
                        PD_baseline_pathlength_T]

    list_baseline_metrics=["Instantaneous Speed",
                        "Interval Speed",
                        "Bias",
                        "Morphwidth",
                        "Midline",
                        "Area",
                        "Angular Speed",
                        "Aspect Ratio",
                        "Kink",
                        "Curve",
                        "Crab",
                        "Pathlength"]
    return list_baseline_Tstats, list_baseline_metrics

In [134]:
def tap_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for tap analysis

    input:
        - by: what to group by "Gene" or "dataset"
    returns:
        - list_baseline_Tstats: to store t-statistics
        - list_baseline_metrics: to store metic names
    """
    recovery_dura=pd.DataFrame(columns = [by,"Recovery Duration"])
    recovery_prob=pd.DataFrame(columns = [by,"Recovery Probability"])
    recovery_speed=pd.DataFrame(columns = [by,"Recovery Speed"])
    init_dura=pd.DataFrame(columns = [by,"Initial Duration"])
    init_prob=pd.DataFrame(columns = [by,"Initial Probability"])
    init_speed=pd.DataFrame(columns = [by,"Initial Speed"])
    final_dura=pd.DataFrame(columns = [by,"Final Duration"])
    final_prob=pd.DataFrame(columns = [by,"Final Probability"])
    final_speed=pd.DataFrame(columns = [by,"Final Speed"])
    hab_dura=pd.DataFrame(columns = [by,"Habituation of Duration"])
    hab_prob=pd.DataFrame(columns = [by,"Habituation of Probability"])
    hab_speed=pd.DataFrame(columns = [by,"Habituation of Speed"])

    list_tap_Tstats = [recovery_dura,
                    recovery_prob,
                    recovery_speed,
                    init_dura,
                    init_prob,
                    init_speed,
                    final_dura,
                    final_prob,
                    final_speed,
                    hab_dura,
                    hab_prob,
                    hab_speed]
    list_tap_metrics = ["recovery_dura",
                        "recovery_prob",
                        "recovery_speed",
                        "init_dura",
                        "init_prob",
                        "init_speed",
                        "final_dura",
                        "final_prob",
                        "final_speed",
                        "habit_dura",
                        "habit_prob",
                        "habit_speed"]
    return list_tap_Tstats, list_tap_metrics

In [135]:
def TTest(Type, DF_ref, output, by=["Gene", "dataset"]):
    """
    Perform two sample t-test for each unique Gene/dataset column in the Df_ref
    input: 
        - a:column name of values 
        - DF_ref:reference dataframe
        - output: output df to store results in 
        - by: what to group by "Gene" or "dataset"
    """
    for a in DF_ref[by].unique():
        Tstat_a =ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele.isin(["XJ1","N2"])][Type],equal_var=False)[0]
        Tstat_g= ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        Tstat = Tstat_a if by=="dataset" else Tstat_g
        row=[a, Tstat]
        output.loc[len(output)]=row
    # print(output)

def do_TTest(by=["Gene", "dataset"], baseline=["true", "false"]):
    """
    Perform TTest function for each unique Gene/dataset column in baseline_output/tap_data
    
    input: 
        - by: what to group by "Gene" or "dataset"
        - baseline: whether or not to use baseline data

    returns: sorted T-statistics dataframe
    """

    if baseline=="true":
        list_Tstats,list_metrics=baseline_metrics(by)
        data = baseline_output
    else:
        list_Tstats,list_metrics=tap_metrics(by)
        data =tap_data
    for x in data[by].unique():
        if x == "N2":
            pass
        else:
            output_gene=data[data[by]==x]
            gene_data=data[data['Date'].isin(output_gene['Date'].unique())]
            gene_data_final=gene_data[gene_data[by].isin(['N2', x])]
            for a,b in zip(list_metrics, list_Tstats):
                TTest(a, gene_data_final, b, by) #calls t test function
    
    PD_Tstats=pd.DataFrame()
    for a in list_Tstats:
        b=a.groupby([by], as_index=False).mean()
        if b.columns.values[1] == list_Tstats[0].columns.values[1]:
            PD_Tstats=b
        else:
            PD_Tstats=PD_Tstats.join(b.iloc[:,1])
            
    PD_Tstats=PD_Tstats.set_index(by)
    return PD_Tstats
            

## T-stat analysis for Baseline data

In [136]:
print(baseline_output.head())

    Time   n  Number  Instantaneous Speed  Interval Speed   Bias  Morphwidth  \
0  490.0  77      40               0.0417          0.0581  0.100      0.1140   
1  490.0  76      40               0.0510          0.0729  0.100      0.1152   
2  490.0  77      40               0.0589          0.0864  0.100      0.1139   
3  490.0  77      40               0.0445          0.0662  0.075      0.1140   
4  490.2  77      40               0.0452          0.0656  0.100      0.1155   

   Midline    Area  Angular Speed  ...  Kink  Curve    Crab  Pathlength  \
0    1.095  0.1472            1.6  ...  40.8   27.0  0.0041       10.33   
1    1.093  0.1483            2.2  ...  40.0   26.5  0.0049       10.33   
2    1.090  0.1462            2.5  ...  39.5   26.8  0.0058       10.33   
3    1.089  0.1464            2.0  ...  40.4   27.3  0.0048       10.33   
4    1.094  0.1482            2.2  ...  40.9   27.3  0.0059       10.33   

   Plate_id      Date     Screen dataset Gene Allele  
0   B0520ab  

### Allele-level T-stat analysis of baseline data

In [137]:
PD_baseline_Tstats_allele = do_TTest("dataset", baseline="true") # get sorted T-statistics DataFrame 

In [138]:
print(PD_baseline_Tstats_allele.head())
PD_baseline_Tstats_allele_sorted=PD_baseline_Tstats_allele.sort_index()


               Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
dataset                                                                      
F53B2.5_ok226           268.193323      138.469656  261.049629  -22.704574   
N2                        0.000000        0.000000    0.000000    0.000000   
cat-1_ok411             189.625948      283.600422  160.997794  -61.093269   
cat-2_e1112             191.587952      165.421172  205.719599   62.800458   
cat-2_n4547             218.091570      210.247050  196.277588  -27.815042   

                  Midline        Area  Angular Speed  Aspect Ratio  \
dataset                                                              
F53B2.5_ok226  -34.961132  -60.981786     179.734246     67.210380   
N2               0.000000    0.000000       0.000000      0.000000   
cat-1_ok411   -280.936078 -230.343493     184.617826    210.825465   
cat-2_e1112     91.326973  151.900546     137.743554    173.340518   
cat-2_n4547    -87.754452 -132.34

### Gene-level T-stat analysis of baseline data

In [139]:
PD_baseline_Tstats=do_TTest("Gene", baseline="true") # get sorted T-statistics DataFrame 

In [140]:
print(PD_baseline_Tstats.head())
PD_baseline_Tstats_sorted=PD_baseline_Tstats.sort_index()


         Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
Gene                                                                   
F53B2.5           268.193323      138.469656  261.049629  -22.704574   
N2                  0.000000        0.000000    0.000000    0.000000   
cat-1             189.625948      283.600422  160.997794  -61.093269   
cat-2             254.122105      204.278433  203.395932   57.815388   
catp-7            -63.282341      -77.819638  -16.661527  -79.237699   

            Midline        Area  Angular Speed  Aspect Ratio        Kink  \
Gene                                                                       
F53B2.5  -34.961132  -60.981786     179.734246     67.210380   17.509099   
N2         0.000000    0.000000       0.000000      0.000000    0.000000   
cat-1   -280.936078 -230.343493     184.617826    210.825465  118.630892   
cat-2     40.167583   65.589547     183.624043    225.448566  164.862510   
catp-7  -142.170469 -117.763188      -7

## T-stat analysis for tap-response data

In [141]:
print(tap_data.head())

   init_dura  init_prob  init_speed  plate      Date Plate_id     Screen  \
0       2.64   0.945946    0.279924      1  20220524  B0520ba  PD_Screen   
1       2.84   0.921569    0.324296      2  20220524  B0520bd  PD_Screen   
2       3.64   0.947368    0.342308      3  20220524  A0520be  PD_Screen   
3       3.32   0.925000    0.348795      4  20220524  A0520bc  PD_Screen   
4       3.02   0.883721    0.332119      5  20220524  C0520bb  PD_Screen   

         dataset     Gene Allele  ...  habit_speed      time  recov_dura  \
0  F53B2.5_ok226  F53B2.5  ok226  ...     0.083399  1189.940        3.72   
1  F53B2.5_ok226  F53B2.5  ok226  ...     0.107957  1189.943        3.11   
2  F53B2.5_ok226  F53B2.5  ok226  ...     0.129027  1189.970        3.65   
3  F53B2.5_ok226  F53B2.5  ok226  ...     0.086841  1189.961        2.62   
4  F53B2.5_ok226  F53B2.5  ok226  ...     0.114514  1189.990        2.90   

    dist  recov_prob  recov_speed  taps  recovery_dura  recovery_prob  \
0  1.090    0

### Allele level T-stat analysis of tap response data

In [142]:
PD_habituation_Tstats_allele = do_TTest("dataset", baseline="false") # get sorted T-statistics DataFrame 

In [143]:
print(PD_habituation_Tstats_allele.head())
PD_habituation_Tstats_allele_sorted=PD_habituation_Tstats_allele.sort_index()


               Recovery Duration  Recovery Probability  Recovery Speed  \
dataset                                                                  
F53B2.5_ok226           0.777695             -1.490039       -2.213869   
N2                      0.000000              0.000000        0.000000   
cat-1_ok411            -4.179564             -1.649154       -1.110592   
cat-2_e1112            -1.006119             -1.485955        1.275922   
cat-2_n4547            -1.014884             -1.521969        1.558420   

               Initial Duration  Initial Probability  Initial Speed  \
dataset                                                               
F53B2.5_ok226          2.229089             0.969676       3.818974   
N2                     0.000000             0.000000       0.000000   
cat-1_ok411            1.299847            -1.617840       5.610974   
cat-2_e1112            1.577622             0.917550      -0.974695   
cat-2_n4547            0.996326             0.785285   

### Gene-level T-stat analysis of Tap response data

In [144]:
PD_habituation_Tstats = do_TTest("Gene", baseline="false") # get sorted T-statistics DataFrame 

In [145]:
print(PD_habituation_Tstats)
PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()


         Recovery Duration  Recovery Probability  Recovery Speed  \
Gene                                                               
F53B2.5           0.777695             -1.490039       -2.213869   
N2                0.000000              0.000000        0.000000   
cat-1            -4.179564             -1.649154       -1.110592   
cat-2            -1.005343             -1.434865        1.293532   
catp-7           -0.422819              0.541592       -0.086352   
cpr-1             1.709376              1.392861        0.314967   
crls-1            0.749123              1.238289       -0.467743   
dat-1            -0.988093             -1.313582        0.793537   
dop-1             1.244071              0.740787       -0.430570   
dop-3            -1.122524             -0.908308        0.428746   
gba-3            -0.887436             -0.309162        1.115400   
gly-5             1.726574              3.532712       -0.208289   
ifg-1             0.824359              4.194692

# Merging t-stat data into one dataset

In [146]:
def pop_cols(combined):
    """
    Reorders columns in the combined dataframe. 
    (pops specific columns["Area","Midline", "Morphwidth","Angular Speed"] and
    reinserts at different positions)

    input:
        combined: dataframe with columns to be reordered

    """
    first_col=combined.pop("Area")
    combined.insert(0,"Area",first_col)

    first_col=combined.pop("Midline")
    combined.insert(0,"Midline",first_col)

    first_col=combined.pop("Morphwidth")
    combined.insert(0,"Morphwidth",first_col)

    first_col=combined.pop("Angular Speed")
    combined.insert(5,"Angular Speed",first_col)

def pop_last(combined):
    """
    Reorders the last three columns of the combined dataframe.
    input:
        combined: dataframe with columns to be reordered

    """
    last_col=combined.pop("Spontaneous Recovery of Response Duration")
    combined.insert(23,"Spontaneous Recovery of Response Duration",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Probability")
    combined.insert(23,"Spontaneous Recovery of Response Probability",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Speed")
    combined.insert(23,"Spontaneous Recovery of Response Speed",last_col)

def rename_columns(df):
    '''
    Renames columns in the input dataframe
    input:
        combined: dataframe with columns to be renamed   
    returns:
        input dataframe with renamed columns 
    '''
    renames = {
        "Habituation of Duration": "Habituation of Response Duration",
        "Habituation of Probability": "Habituation of Respones Probability",
        "Habituation of Speed": "Habituation of Response Speed",
        "Initial Duration": "Initial Response Duration",
        "Initial Probability": "Initial Response Probability",
        "Initial Speed": "Initial Response Speed",
        "Final Duration": "Final Response Duration",
        "Final Probability": "Final Response Probability",
        "Final Speed": "Final Response Speed",
        "Recovery Duration": "Spontaneous Recovery of Response Duration",
        "Recovery Probability": "Spontaneous Recovery of Response Probability",
        "Recovery Speed": "Spontaneous Recovery of Response Speed"
    }
    return df.rename(columns=renames)

def merge_Tstats(baseline, habituation, by=["Gene", "dataset"]):
    """
    merge two dataframes based on the Gene/dataset
    normalize the merged dataframe and then return it with melted version

    input:
        - baseline: baseline dataframe to merge
        - habituation: habituation dataframe to merge
        - by: what to group by "Gene" or "dataset"
    """

    #merge baseline and habituation data
    combined_Tstats = pd.merge(baseline, habituation, on=[by], how='left')
    combined_Tstats = combined_Tstats.sort_index() # sort by index
    print("combined_Tstats")
    print(combined_Tstats)

    #normalise combined dataframe by subtracting mean and div by sd
    combined_Tstats_normalized = (combined_Tstats-combined_Tstats.mean())/combined_Tstats.std()
    # if by=="Gene" :
    print(combined_Tstats_normalized[combined_Tstats_normalized.index=="N2"])
    combined_Tstats_normalized_2 = combined_Tstats_normalized-combined_Tstats_normalized[combined_Tstats_normalized.index=="N2"].squeeze()  
    print("combined Tstats normalized N2 data")
    print(combined_Tstats_normalized_2[combined_Tstats_normalized_2.index=="N2"])

    # else:
    #     print(combined_Tstats_normalized[combined_Tstats_normalized.index=="N2_XJ1"])
    #     combined_Tstats_normalized_2 = combined_Tstats_normalized-combined_Tstats_normalized[combined_Tstats_normalized.index=="N2_XJ1"].squeeze()
    #     print("combined Tstats normalized N2_XJ1 data")
    #     print(combined_Tstats_normalized_2[combined_Tstats_normalized_2.index=="N2_XJ1"])


    print("combined_Tstats_normalize")
    print(combined_Tstats_normalized)

    pop_cols(combined_Tstats) #reorder columns
    print("combined_Tstats")
    print(combined_Tstats)

    #rename columns of combined and normalized df
    combined_Tstats=rename_columns(combined_Tstats)
    combined_Tstats_normalized_2=rename_columns(combined_Tstats_normalized_2)

    print("combined_Tstats_normalize_2")
    print(combined_Tstats_normalized_2)
    
    pop_cols(combined_Tstats_normalized_2) #reorder columns
    pop_last(combined_Tstats_normalized_2) #reorder columns

    print("combined_Tstats_normalize_2")
    print(combined_Tstats_normalized_2)

    # Melt the combined dataframe
    combined_Tstats_melted=combined_Tstats.reset_index()
    combined_Tstats_melted=pd.melt(combined_Tstats_melted, id_vars=[by],
                                var_name='Metric',
                                value_name='T_score')
    # Sort the melted dataframe by T_score
    combined_Tstats_melted_sorted=combined_Tstats_melted.sort_values(by=['T_score'])
    print("combined_Tstats_melted_sorted")
    print(combined_Tstats_melted_sorted)

    # Melt the normalized dataframe
    combined_Tstats_normalized_melted=combined_Tstats_normalized_2.reset_index()
    combined_Tstats_normalized_melted=pd.melt(combined_Tstats_normalized_melted, id_vars=[by],
                                                   var_name='Metric',
                                                   value_name='T_score')
    print("combined_Tstats_normalized_melted")
    print(combined_Tstats_normalized_melted)

    #add Screen column to df and its melted version
    combined_Tstats_normalized_2['Screen']=Screen
    combined_Tstats_normalized_melted['Screen']=Screen

    return combined_Tstats_normalized_2,combined_Tstats_normalized_melted

## Gene-level

In [147]:
combined_Tstats_normalize_2, combined_Tstats_normalized_melted = merge_Tstats(PD_baseline_Tstats,PD_habituation_Tstats, "Gene")


combined_Tstats
         Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
Gene                                                                   
F53B2.5           268.193323      138.469656  261.049629  -22.704574   
N2                  0.000000        0.000000    0.000000    0.000000   
cat-1             189.625948      283.600422  160.997794  -61.093269   
cat-2             254.122105      204.278433  203.395932   57.815388   
catp-7            -63.282341      -77.819638  -16.661527  -79.237699   
cpr-1             -14.091782      -17.274725  -60.383353  -37.234241   
crls-1           -113.760901      -87.324605  -58.781751  -48.276501   
dat-1             124.536042       61.101402  111.971307   40.572013   
dop-1              -4.955540      -19.230744   31.682482  -20.009550   
dop-3              58.852105       69.282305   28.361126   -6.626313   
gba-3             -35.847821      -13.009577  -25.551753  -28.375385   
gly-5             -32.931185      -26.478952    

## Allele level

In [148]:
combined_Tstats_normalize_allele_2, combined_Tstats_normalized_melted_allele = merge_Tstats(PD_baseline_Tstats_allele,PD_habituation_Tstats_allele, "dataset")

combined_Tstats
                 Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
dataset                                                                        
F53B2.5_ok226             268.193323      138.469656  261.049629  -22.704574   
N2                          0.000000        0.000000    0.000000    0.000000   
cat-1_ok411               189.625948      283.600422  160.997794  -61.093269   
cat-2_e1112               191.587952      165.421172  205.719599   62.800458   
cat-2_n4547               218.091570      210.247050  196.277588  -27.815042   
cat-2_tm2261              147.778921      122.294942   87.886529   94.747046   
catp-7_tm8556             -63.282341      -77.819638  -16.661527  -79.237699   
cpr-1_ok1344              -14.091782      -17.274725  -60.383353  -37.234241   
crls-1_tm2575            -113.760901      -87.324605  -58.781751  -48.276501   
dat-1_ok157               124.536042       61.101402  111.971307   40.572013   
dop-1_vs101             

In [149]:
print(tap_output)

         time  dura   dist      prob     speed  plate      Date Plate_id  \
0     599.983  2.64  0.739  0.945946  0.279924      1  20220524  B0520ba   
1     609.995  2.66  0.834  0.857143  0.313534      1  20220524  B0520ba   
2     619.933  2.48  0.759  0.833333  0.306048      1  20220524  B0520ba   
3     629.932  2.19  0.649  0.901961  0.296347      1  20220524  B0520ba   
4     639.991  2.06  0.589  0.787234  0.285922      1  20220524  B0520ba   
..        ...   ...    ...       ...       ...    ...       ...      ...   
150   859.983  1.89  0.281  0.058824  0.148677      5  20220513  A0509be   
151   869.999  1.36  0.214  0.058824  0.157353      5  20220513  A0509be   
152   879.997  5.38  0.856  0.025641  0.159108      5  20220513  A0509be   
153   889.992  0.12  0.011  0.027027  0.091667      5  20220513  A0509be   
154  1189.990  2.24  0.355  0.148148  0.158482      5  20220513  A0509be   

        Screen  taps        dataset     Gene Allele  
0    PD_Screen   1.0  F53B2.5_ok2

In [150]:
print(combined_MSD)

       Gene  Instantaneous Speed-mean  Instantaneous Speed-count  \
0   F53B2.5                  0.086278                          5   
1        N2                  0.000000                         40   
2     cat-1                  0.065821                          5   
3     cat-2                  0.055155                         15   
4    catp-7                 -0.013505                          5   
5     cpr-1                  0.004338                          5   
6    crls-1                 -0.020205                          5   
7     dat-1                  0.029433                          5   
8     dop-1                  0.012138                          5   
9     dop-3                  0.042760                          5   
10    gba-3                  0.002792                          5   
11    gly-5                 -0.001453                          4   
12    ifg-1                  0.019292                          5   
13   immt-1                  0.002092           

,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,...,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,dataset,Gene,Allele
0,490.0,92,34,0.0764,0.0735,0.182,0.0819,1.122,0.11847,5.2,...,48.8,30.0,0.0140,4.457,A0419,20220419,Glia_Genes_Screen,N2,N2,N2
1,490.0,92,34,0.0637,0.0641,0.152,0.0821,1.122,0.11850,4.7,...,48.1,29.8,0.0114,4.460,A0419,20220419,Glia_Genes_Screen,N2,N2,N2
2,490.0,92,34,0.0639,0.0698,0.182,0.0818,1.122,0.11804,4.0,...,48.6,29.4,0.0096,4.460,A0419,20220419,Glia_Genes_Screen,N2,N2,N2
3,490.0,92,34,0.0608,0.0644,0.212,0.0820,1.130,0.11865,3.4,...,49.0,29.7,0.0098,4.465,A0419,20220419,Glia_Genes_Screen,N2,N2,N2
4,490.2,92,34,0.0978,0.1282,0.242,0.0817,1.128,0.11835,6.1,...,48.1,29.4,0.0142,4.465,A0419,20220419,Glia_Genes_Screen,N2,N2,N2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092305,590.0,72,30,0.0664,0.0890,0.321,0.1094,1.179,0.15150,4.2,...,56.1,32.0,0.0098,5.450,A0905,20220905,Glia_Genes_Screen,ced-12_n3261,ced-12,n3261
1092306,590.0,72,30,0.0740,0.0895,0.321,0.1091,1.178,0.15120,5.0,...,55.9,32.0,0.0134,5.453,A0905,20220905,Glia_Genes_Screen,ced-12_n3261,ced-12,n3261
1092307,590.0,73,30,0.0836,0.1078,0.286,0.1098,1.175,0.15160,5.8,...,55.6,32.0,0.0163,5.453,A0905,20220905,Glia_Genes_Screen,ced-12_n3261,ced-12,n3261
1092308,590.0,73,30,0.0920,0.1235,0.321,0.1090,1.182,0.15140,6.4,...,56.2,31.9,0.0132,5.457,A0905,20220905,Glia_Genes_Screen,ced-12_n3261,ced-12,n3261


## sqlite3 upload data

## A janky way to add data and update the sql 

1. Read table to pd.DataFrame
2. Add new data to pd.DataFrame
3. Replace old table with newly updated pd.DataFrame

In [152]:
# USE THIS CELL TO UPDATE ALL THE NEED TALBES (Also have baseline_output on the second line)

conn=sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/data_updated.db')
tap_output.to_sql('tap_response_data', conn, if_exists='append', index=False)
baseline_output.to_sql('tap_baseline_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='append', index=False)
combined_MSD.to_sql('gene_MSD', conn, if_exists='append', index=False)
allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='append', index=False)
# combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)
print(conn.total_changes)
conn.close()


# Want to test edge cases of pd.to_sql functionality#############

295564


In [ ]:
#tstat_gene_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM tstat_gene_data", conn)
conn.close()
grouped = tstat_gene.groupby('Gene').agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['Gene', 'Screen']}
    ).reset_index()


def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby('Gene')['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("tstat_gene_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [ ]:
#tstat_allele_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM tstat_allele_data", conn)
conn.close()
grouped = tstat_gene.groupby('dataset').agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['dataset', 'Screen']}
    ).reset_index()


def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby('dataset')['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("tstat_allele_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [61]:
#gene_profile_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM gene_profile_data", conn)
conn.close()
print(tstat_gene)
grouped = tstat_gene.groupby(['Gene','Metric']).agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['Gene', 'Screen','Metric']}
    ).reset_index()
grouped['Screen'] = tstat_gene.groupby(['Gene','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)
print(grouped)
def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby(['Gene','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("gene_profile_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

['tap_response_data', 'gene_profile_data', 'allele_profile_data', 'gene_MSD', 'allele_MSD', 'tap_baseline_data', 'tstat_gene_data', 'tstat_allele_data']
          Gene                         Metric   T_score             Screen
0     C29E4.10                     Morphwidth -0.641065          PD_Screen
1     F32D8.13                     Morphwidth  0.769580          PD_Screen
2      F35C8.1                     Morphwidth  0.056716          PD_Screen
3      F53B2.5                     Morphwidth -0.299505          PD_Screen
4     F54B11.5                     Morphwidth -0.825061          PD_Screen
...        ...                            ...       ...                ...
9835    unc-13  Habituation of Response Speed       NaN  G-Proteins_Screen
9836     unc-2  Habituation of Response Speed       NaN  G-Proteins_Screen
9837    unc-73  Habituation of Response Speed       NaN  G-Proteins_Screen
9838   unknown  Habituation of Response Speed       NaN  G-Proteins_Screen
9839    zig-12  Habitu

In [ ]:
#allele_profile_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM allele_profile_data", conn)
conn.close()
print(tstat_gene)
grouped = tstat_gene.groupby(['dataset','Metric']).agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['dataset', 'Screen','Metric']}
    ).reset_index()
grouped['Screen'] = tstat_gene.groupby(['dataset','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)
print(grouped)
def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby(['dataset','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("allele_profile_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
#allele_MSD
# conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
# tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
# tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
# print(tables)
# tstat_gene = pd.read_sql_query(f"SELECT * FROM allele_MSD", conn)
# conn.close()
# print(tstat_gene)
# def weighted_avg(df, col_mean, col_count):
#     return (df[col_mean] * df[col_count]).sum() / df[col_count].sum()

# grouped = tstat_gene.groupby('dataset').agg(
#     {col: lambda x: weighted_avg(x, col, col.replace('-mean', '-count')) 
#      for col in tstat_gene.columns if col.endswith('-mean')}
# ).reset_index()

# count_cols = [col for col in tstat_gene.columns if col.endswith('-count')]
# for col in count_cols:
#     grouped[col] = tstat_gene.groupby('dataset')[col].sum().values

# grouped['Screen'] = tstat_gene.groupby(['dataset'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)
# print(grouped)

# def compute_ci95(df, metric):
#     ci95_hi = []
#     ci95_lo = []
#     for i in df.index:
#         m = df[f'{metric}-mean'].loc[i]
#         c = df[f'{metric}-count'].loc[i]
#         s = df[f'{metric}-sem'].loc[i]
#         ci_hi, ci_lo = stats.t.interval(0.95, c-1, loc=m, scale=s)
#         ci95_hi.append(ci_hi)
#         ci95_lo.append(ci_lo)
#     df[f'{metric}-ci95_hi'] = ci95_hi
#     df[f'{metric}-ci95_lo'] = ci95_lo
#     return df

# # def transform_to_list(row):
# #     screens = row.split(', ')
# #     return list(set(screens))

# # # Aggregate the Screen column into a list
# # grouped['Screen'] = tstat_gene.groupby(['dataset'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # # Optionally, convert the Screen lists to a string representation
# # grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
# # grouped['Screen']= grouped['Screen'].apply(transform_to_list)
# # grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
# # conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
# metrics = set([col.split('-')[0] for col in tstat_gene.columns if col.endswith('-mean')])
# for metric in metrics:
#     grouped = compute_ci95(grouped, metric)

# print(grouped)
# # Insert the aggregated data into the new table
# # grouped.to_sql("allele_MSD", conn, if_exists='replace', index=False)

# # Commit the changes and close the connection
# conn.commit()
# conn.close()

In [60]:
grouped.Screen.unique()

array(['ASD_Screen', 'PD_Screen',
       'ASD_Screen, G-Proteins_Screen, PD_Screen, Glia_Genes_Screen',
       'G-Proteins_Screen', 'Glia_Genes_Screen', 'PD_Screen, ASD_Screen',
       'G-Proteins_Screen, ASD_Screen', 'G-Proteins_Screen, PD_Screen',
       'G-Proteins_Screen, Glia_Genes_Screen'], dtype=object)

In [47]:
tstat_gene

,Gene,Metric,T_score,Screen
0,F53B2.5,Morphwidth,-0.195640,PD_Screen
1,N2,Morphwidth,0.000000,PD_Screen
2,cat-1,Morphwidth,-0.526426,PD_Screen
3,cat-2,Morphwidth,0.498182,PD_Screen
4,catp-7,Morphwidth,-0.682773,PD_Screen
...,...,...,...,...
499,pdr-1,Habituation of Response Speed,0.012246,PD_Screen
500,rme-8,Habituation of Response Speed,0.868745,PD_Screen
501,sipa-1,Habituation of Response Speed,-1.529130,PD_Screen
502,snb-5,Habituation of Response Speed,0.359162,PD_Screen


In [ ]:
# cursor = conn.cursor()

# cursor.execute("SELECT COUNT(*) FROM table_name WHERE primary_key_column = ?", (value,))
# exists = cursor.fetchone()[0] > 0

# if exists:
#     print("Primary key already exists in the table.")
# else:
#     print("Primary key does not exist in the table.")

In [ ]:
# conn.close()

## Done!